# Avito ML cup 2025 - Персональные рекомендации товаров

# Загрузка данных

In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
# Скачиваем нужные библиотеки
!pip install polars==1.25.2 >> _
!pip install implicit >> _

In [4]:
# Содаем папку data для хранения датасетов
!mkdir data

Подпапка или файл data уже существует.


In [5]:
# Скачиваем датасеты
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/clickstream.pq -O data/clickstream.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/test_users.pq -O data/test_users.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/cat_features.pq -O data/cat_features.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/text_features.pq -O data/text_features.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/events.pq -O data/events.pq >> _


In [6]:
# Подключаем необходимые библиотеки
from datetime import timedelta
import polars as pl
import implicit

In [7]:
# Загрузим все подготовленные данные из папки data/ в DataFrame Polars.
DATA_DIR = 'data/'

df_test_users = pl.read_parquet(f'{DATA_DIR}/test_users.pq')
df_clickstream = pl.read_parquet(f'{DATA_DIR}/clickstream.pq')
df_cat_features = pl.read_parquet(f'{DATA_DIR}/cat_features.pq')
df_text_features = pl.read_parquet(f'{DATA_DIR}/text_features.pq')
df_event = pl.read_parquet(f'{DATA_DIR}/events.pq')

**Описание данных:**
1. `df_test_users` - список пользователей для предсказаний
2. `df_clickstream` - история кликов пользователей
3. `df_cat_features` - категориальные признаки
4. `df_text_features` - текстовые признаки
5. `df_event` - события

**Описание столбцов:**
- `cookie` - пользователь
- `item` - id объявлений
- `event` - код событий, которые произошли
- `event_date` - дата события
- `platform` - 6 вариантов платформ
- `surface` - экран, с которого было взаимодействие
- `node` - id группы товара
- `is_contact` - контактное ли событие

# Исследовательский анализ данных

## Изучим и опишем данные из таблицы df_test_users

In [8]:
df_test_users, 
df_test_users['cookie'].describe(), 
len(df_test_users['cookie'].unique()), 
df_test_users['cookie'].hist()

(shape: (92_319, 1)
 ┌────────┐
 │ cookie │
 │ ---    │
 │ i64    │
 ╞════════╡
 │ 52564  │
 │ 105000 │
 │ 57152  │
 │ 87303  │
 │ 37755  │
 │ …      │
 │ 78910  │
 │ 64750  │
 │ 118889 │
 │ 131    │
 │ 37487  │
 └────────┘,
 shape: (9, 2)
 ┌────────────┬──────────────┐
 │ statistic  ┆ value        │
 │ ---        ┆ ---          │
 │ str        ┆ f64          │
 ╞════════════╪══════════════╡
 │ count      ┆ 92319.0      │
 │ null_count ┆ 0.0          │
 │ mean       ┆ 75192.415429 │
 │ std        ┆ 43297.982566 │
 │ min        ┆ 1.0          │
 │ 25%        ┆ 37653.0      │
 │ 50%        ┆ 75255.0      │
 │ 75%        ┆ 112733.0     │
 │ max        ┆ 149999.0     │
 └────────────┴──────────────┘,
 92319,
 shape: (10, 3)
 ┌────────────┬──────────────────────┬───────┐
 │ breakpoint ┆ category             ┆ count │
 │ ---        ┆ ---                  ┆ ---   │
 │ f64        ┆ cat                  ┆ u32   │
 ╞════════════╪══════════════════════╪═══════╡
 │ 15000.8    ┆ (-148.998, 15000.8]

**Как видно из таблице выше:** 
- в тестовом файле 92 319 пользователей с cookies

- для каждого нужно определить группы nodes

- эти группы не должны быть теми, которые он уже кликал

- нужно определить, на какие группы товаров пользователи вероятнее всего откликнутся, из тех, с которыми они еще не взаимодействовали

- рекомендуется по 10 групп выдавать каждому пользователю как ответ

## Изучим и опишем данные из таблицы df_clickstream

In [9]:
df_clickstream.describe()

statistic,cookie,item,event,event_date,platform,surface,node
str,f64,f64,f64,str,f64,f64,f64
"""count""",6.8806152e7,6.8806152e7,6.8806152e7,"""68806152""",6.8806152e7,6.8806152e7,6.8806152e7
"""null_count""",0.0,0.0,0.0,"""0""",0.0,0.0,0.0
"""mean""",74884.102841,1.4402e7,16.29883,"""2025-02-01 15:21:43.914605""",2.374055,6.051714,165425.273495
"""std""",43250.599321,8.3145e6,2.188569,null,0.776999,4.695714,88418.121967
"""min""",0.0,0.0,0.0,"""2025-01-10 00:00:00""",0.0,0.0,1.0
"""25%""",37353.0,7.20358e6,17.0,"""2025-01-21 16:02:41""",2.0,2.0,116118.0
"""50%""",74884.0,1.440721e7,17.0,"""2025-02-01 23:37:08""",2.0,3.0,153937.0
"""75%""",112113.0,2.1605283e7,17.0,"""2025-02-12 16:12:00""",3.0,11.0,214338.0
"""max""",149999.0,2.8804867e7,19.0,"""2025-02-23 00:00:00""",6.0,18.0,424068.0


In [10]:
# Выведем первые 100 товаров, отсортированных по значению
print(df_clickstream['item'].sort().head(100).to_list())

[0, 0, 1, 1, 3, 3, 5, 6, 7, 8, 9, 10, 10, 10, 10, 10, 10, 10, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 16, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 20, 20, 22, 23, 24, 25, 25, 26, 26, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28]


In [11]:
# Создадим функцию для анализа распределения частот в столбцах
def calc_dist_param(column, data=df_clickstream, verbose=True):
    clicks_count = data[column].value_counts().sort(by=['count'])
    
    if verbose:
        print(f'CLICKS COUNT count distr (value_counts) for column \"{column}\":')
        display(clicks_count)
        print('===DESCRIBE===')
        display(clicks_count.describe())
        print('===HIST===')
        display(clicks_count['count'].hist())
        
    return clicks_count

In [12]:
# Распределение кол-ва кликов по пользователям
calc_dist_param('cookie')

CLICKS COUNT count distr (value_counts) for column "cookie":


cookie,count
i64,u32
59442,1
111720,1
48324,1
88073,1
45563,1
…,…
31003,16341
116470,18030
6003,21680


===DESCRIBE===


statistic,cookie,count
str,f64,f64
"""count""",134294.0,134294.0
"""null_count""",0.0,0.0
"""mean""",75065.303163,512.354625
"""std""",43299.301473,823.573902
"""min""",0.0,1.0
"""25%""",37570.0,67.0
"""50%""",75042.0,231.0
"""75%""",112571.0,625.0
"""max""",149999.0,22864.0


===HIST===


breakpoint,category,count
f64,cat,u32
2287.3,"""(-21.863, 2287.3]""",129639
4573.6,"""(2287.3, 4573.6]""",3817
6859.9,"""(4573.6, 6859.9]""",601
9146.2,"""(6859.9, 9146.2]""",138
11432.5,"""(9146.2, 11432.5]""",59
13718.8,"""(11432.5, 13718.8]""",22
16005.1,"""(13718.8, 16005.1]""",12
18291.4,"""(16005.1, 18291.4]""",3
20577.7,"""(18291.4, 20577.7]""",0


cookie,count
i64,u32
59442,1
111720,1
48324,1
88073,1
45563,1
…,…
31003,16341
116470,18030
6003,21680


In [13]:
# Распределение кол-ва кликов по товарам
calc_dist_param('item')

CLICKS COUNT count distr (value_counts) for column "item":


item,count
i64,u32
9671567,1
27041484,1
24848736,1
20659765,1
3695198,1
…,…
16454256,2116
17649863,2270
22565184,2530


===DESCRIBE===


statistic,item,count
str,f64,f64
"""count""",2.2646691e7,2.2646691e7
"""null_count""",0.0,0.0
"""mean""",1.4402e7,3.038243
"""std""",8.3149e6,6.415441
"""min""",0.0,1.0
"""25%""",7.201417e6,1.0
"""50%""",1.4402191e7,1.0
"""75%""",2.160266e7,3.0
"""max""",2.8804867e7,2642.0


===HIST===


breakpoint,category,count
f64,cat,u32
265.1,"""(-1.641, 265.1]""",22645787
529.2,"""(265.1, 529.2]""",753
793.3,"""(529.2, 793.3]""",101
1057.4,"""(793.3, 1057.4]""",31
1321.5,"""(1057.4, 1321.5]""",5
1585.6,"""(1321.5, 1585.6]""",7
1849.7,"""(1585.6, 1849.7]""",1
2113.8,"""(1849.7, 2113.8]""",1
2377.9,"""(2113.8, 2377.9]""",2


item,count
i64,u32
9671567,1
27041484,1
24848736,1
20659765,1
3695198,1
…,…
16454256,2116
17649863,2270
22565184,2530


In [14]:
# Распределение кол-ва кликов по платформам
calc_dist_param('platform')

CLICKS COUNT count distr (value_counts) for column "platform":


platform,count
i64,u32
6,202
4,1669
1,10299
5,1980699
0,2253252
3,24307843
2,40252188


===DESCRIBE===


statistic,platform,count
str,f64,f64
"""count""",7.0,7.0
"""null_count""",0.0,0.0
"""mean""",3.0,9.8295e6
"""std""",2.160247,1.6040e7
"""min""",0.0,202.0
"""25%""",2.0,10299.0
"""50%""",3.0,1.980699e6
"""75%""",5.0,2.4307843e7
"""max""",6.0,4.0252188e7


===HIST===


breakpoint,category,count
f64,cat,u32
4025400.6,"""(-40049.986, 4025400.6]""",5
8050599.2,"""(4025400.6, 8050599.2]""",0
1.2076e7,"""(8050599.2, 1.2076e7]""",0
1.6101e7,"""(1.2076e7, 1.6101e7]""",0
2.0126195e7,"""(1.6101e7, 2.0126195e7]""",0
2.4151e7,"""(2.0126195e7, 2.4151e7]""",0
2.8177e7,"""(2.4151e7, 2.8177e7]""",1
3.2202e7,"""(2.8177e7, 3.2202e7]""",0
3.6227e7,"""(3.2202e7, 3.6227e7]""",0


platform,count
i64,u32
6,202
4,1669
1,10299
5,1980699
0,2253252
3,24307843
2,40252188


In [15]:
# Распределение кол-ва кликов по экранам, с которых были взаимодействия
calc_dist_param('surface')

CLICKS COUNT count distr (value_counts) for column "surface":


surface,count
i64,u32
12,74
16,647
18,2180
1,7412
0,7608
…,…
15,2799442
3,3943848
5,3949883


===DESCRIBE===


statistic,surface,count
str,f64,f64
"""count""",19.0,19.0
"""null_count""",0.0,0.0
"""mean""",9.0,3.6214e6
"""std""",5.627314,8.5725e6
"""min""",0.0,74.0
"""25%""",5.0,12925.0
"""50%""",9.0,71475.0
"""75%""",14.0,2.799442e6
"""max""",18.0,3.319996e7


===HIST===


breakpoint,category,count
f64,cat,u32
3320062.6,"""(-33125.886, 3320062.6]""",15
6640051.2,"""(3320062.6, 6640051.2]""",2
9960039.8,"""(6640051.2, 9960039.8]""",0
1.3280e7,"""(9960039.8, 1.3280e7]""",0
1.6600017e7,"""(1.3280e7, 1.6600017e7]""",0
1.9920e7,"""(1.6600017e7, 1.9920e7]""",0
2.3240e7,"""(1.9920e7, 2.3240e7]""",1
2.6560e7,"""(2.3240e7, 2.6560e7]""",0
2.9880e7,"""(2.6560e7, 2.9880e7]""",0


surface,count
i64,u32
12,74
16,647
18,2180
1,7412
0,7608
…,…
15,2799442
3,3943848
5,3949883


In [16]:
# Сгруппируем записи по платформе и экранам
(
    df_clickstream
    .group_by(["platform", "surface"])  # Группируем по нескольким столбцам
    .agg(pl.col("node").count().alias("count"))  # Подсчитываем количество записей в каждой группе
).sort(by=['platform'])

platform,surface,count
i64,i64,u32
0,12,2
0,4,3120
0,13,2061
0,3,117619
0,10,4640
…,…,…
5,12,1
5,5,252058
6,5,198


In [19]:
# Распределение кол-ва кликов по группам товаров
node_clicks_count = calc_dist_param('node')

CLICKS COUNT count distr (value_counts) for column "node":


node,count
u32,u32
358337,1
50520,1
405706,1
193105,1
8958,1
…,…
71511,606865
71546,653126
71514,780292


===DESCRIBE===


statistic,node,count
str,f64,f64
"""count""",408474.0,408474.0
"""null_count""",0.0,0.0
"""mean""",211451.832327,168.446834
"""std""",122084.237383,5244.534765
"""min""",1.0,1.0
"""25%""",106081.0,3.0
"""50%""",210711.0,8.0
"""75%""",316972.0,23.0
"""max""",424068.0,2.184712e6


===HIST===


breakpoint,category,count
f64,cat,u32
218472.1,"""(-2183.711, 218472.1]""",408452
436943.2,"""(218472.1, 436943.2]""",16
655414.3,"""(436943.2, 655414.3]""",3
873885.4,"""(655414.3, 873885.4]""",1
1092356.5,"""(873885.4, 1092356.5]""",1
1310827.6,"""(1092356.5, 1310827.6]""",0
1529298.7,"""(1310827.6, 1529298.7]""",0
1747769.8,"""(1529298.7, 1747769.8]""",0
1.9662e6,"""(1747769.8, 1.9662e6]""",0


In [22]:
# Выведем 1000 наиболее кликабельных групп товаров
top_1000_nodes = node_clicks_count.tail(1000)['node']
print('TOP-1000 nodes:', top_1000_nodes)

TOP-1000 nodes: shape: (1_000,)
Series: 'node' [u32]
[
	153740
	336245
	156093
	196448
	214253
	…
	71511
	71546
	71514
	170538
	151453
]


In [23]:
node_clicks_count['count'].quantile(.95), 408474*0.05

(186.0, 20423.7)

**Как видно из таблиц выше:**
- На большинство объявлений кликали 1-3 раза, максимальное кол-во кликов на одно объявление - 2 642 раза
- Общая продолжительность отслеживания кликов - 1.5 месяца
- Наиболее популярные платформы - 2, 3, 0, 5
- Чаще всего кликали с экранов 2, 11, 5,3,15
- У большинства групп товаров малое кол-во кликов - меньше 23, такие группы рекомендовать не стоит

## Изучим и опишем данные из таблицы df_cat_features

In [24]:
df_cat_features

item,location,category,clean_params,node
i64,i64,i64,str,u32
9,8385,57,"""[{""attr"":1157,""value"":664427},…",194747
17,2707,35,"""[{""attr"":2140,""value"":501466},…",352905
144,8383,8,"""[{""attr"":802,""value"":35791},{""…",17188
202,5397,57,"""[{""attr"":1157,""value"":490527},…",194766
236,2105,64,"""[{""attr"":112,""value"":420797},{…",153951
…,…,…,…,…
28804461,24,35,"""[{""attr"":2140,""value"":364348},…",326792
28804502,2305,51,"""[{""attr"":4622,""value"":171723},…",401208
28804563,2348,0,"""[{""attr"":914,""value"":93691},{""…",13974


In [25]:
df_cat_features['item'].n_unique(), 
df_cat_features['location'].n_unique(), 
df_cat_features['category'].n_unique(), 
df_cat_features['node'].n_unique()

(22646691, 4823, 53, 408474)

In [26]:
df_cat_features.describe()

statistic,item,location,category,clean_params,node
str,f64,f64,f64,str,f64
"""count""",2.2646691e7,2.264669e7,2.264669e7,"""22646691""",2.2646691e7
"""null_count""",0.0,1.0,1.0,"""0""",0.0
"""mean""",1.4402e7,4349.799844,35.778449,null,173768.117695
"""std""",8.3149e6,2580.792016,15.774887,null,93004.255815
"""min""",0.0,0.0,0.0,"""[]""",1.0
"""25%""",7.201417e6,2348.0,24.0,null,116118.0
"""50%""",1.4402191e7,3707.0,35.0,null,170538.0
"""75%""",2.160266e7,6773.0,51.0,null,229443.0
"""max""",2.8804867e7,9579.0,64.0,"""[{""attr"":979,""value"":795287},{…",424068.0


In [29]:
# Распределение кол-ва объявлений по локациям
locations_count = calc_dist_param('location', data=df_cat_features)

CLICKS COUNT count distr (value_counts) for column "location":


location,count
i64,u32
8507,1
5361,1
1962,1
3025,1
2552,1
…,…
4283,367549
8205,447617
8383,582408


===DESCRIBE===


statistic,location,count
str,f64,f64
"""count""",4822.0,4823.0
"""null_count""",1.0,0.0
"""mean""",4783.462256,4695.561062
"""std""",2766.026873,70148.581565
"""min""",0.0,1.0
"""25%""",2417.0,17.0
"""50%""",4748.0,217.0
"""75%""",7170.0,723.0
"""max""",9579.0,4.305927e6


===HIST===


breakpoint,category,count
f64,cat,u32
430593.6,"""(-4304.926, 430593.6]""",4819
861186.2,"""(430593.6, 861186.2]""",2
1.2918e6,"""(861186.2, 1.2918e6]""",0
1722371.4,"""(1.2918e6, 1722371.4]""",1
2.152964e6,"""(1722371.4, 2.152964e6]""",0
2.5836e6,"""(2.152964e6, 2.5836e6]""",0
3.0141e6,"""(2.5836e6, 3.0141e6]""",0
3444741.8,"""(3.0141e6, 3444741.8]""",0
3875334.4,"""(3444741.8, 3875334.4]""",0


In [30]:
locations_count['count'].hist()

breakpoint,category,count
f64,cat,u32
430593.6,"""(-4304.926, 430593.6]""",4819
861186.2,"""(430593.6, 861186.2]""",2
1.2918e6,"""(861186.2, 1.2918e6]""",0
1722371.4,"""(1.2918e6, 1722371.4]""",1
2.152964e6,"""(1722371.4, 2.152964e6]""",0
2.5836e6,"""(2.152964e6, 2.5836e6]""",0
3.0141e6,"""(2.5836e6, 3.0141e6]""",0
3444741.8,"""(3.0141e6, 3444741.8]""",0
3875334.4,"""(3444741.8, 3875334.4]""",0


In [31]:
# Распределение кол-ва объявлений по категориям
calc_dist_param('category', data=df_cat_features)

CLICKS COUNT count distr (value_counts) for column "category":


category,count
i64,u32
null,1
60,2
2,2421
11,6423
47,32284
…,…
40,1120838
24,1134356
19,1280756


===DESCRIBE===


statistic,category,count
str,f64,f64
"""count""",52.0,53.0
"""null_count""",1.0,0.0
"""mean""",32.0,427296.056604
"""std""",19.247613,633616.931597
"""min""",0.0,1.0
"""25%""",15.0,94309.0
"""50%""",32.0,167266.0
"""75%""",48.0,520896.0
"""max""",64.0,3.693923e6


===HIST===


breakpoint,category,count
f64,cat,u32
369393.2,"""(-3692.922, 369393.2]""",35
738785.4,"""(369393.2, 738785.4]""",7
1108177.6,"""(738785.4, 1108177.6]""",6
1477569.8,"""(1108177.6, 1477569.8]""",3
1.846962e6,"""(1477569.8, 1.846962e6]""",0
2216354.2,"""(1.846962e6, 2216354.2]""",0
2585746.4,"""(2216354.2, 2585746.4]""",1
2955138.6,"""(2585746.4, 2955138.6]""",0
3.3245e6,"""(2955138.6, 3.3245e6]""",0


category,count
i64,u32
null,1
60,2
2,2421
11,6423
47,32284
…,…
40,1120838
24,1134356
19,1280756


In [32]:
# Распределение кол-ва объявлений по группам товаров
nodes_count = calc_dist_param('node', data=df_cat_features)

CLICKS COUNT count distr (value_counts) for column "node":


node,count
u32,u32
423357,1
178900,1
163734,1
174008,1
325100,1
…,…
71546,136767
71511,146887
71514,185301


===DESCRIBE===


statistic,node,count
str,f64,f64
"""count""",408474.0,408474.0
"""null_count""",0.0,0.0
"""mean""",211451.832327,55.442185
"""std""",122084.237383,1355.580579
"""min""",1.0,1.0
"""25%""",106081.0,1.0
"""50%""",210711.0,3.0
"""75%""",316972.0,8.0
"""max""",424068.0,463451.0


===HIST===


breakpoint,category,count
f64,cat,u32
46346.0,"""(-462.45, 46346.0]""",408433
92691.0,"""(46346.0, 92691.0]""",29
139036.0,"""(92691.0, 139036.0]""",8
185381.0,"""(139036.0, 185381.0]""",2
231726.0,"""(185381.0, 231726.0]""",0
278071.0,"""(231726.0, 278071.0]""",0
324416.0,"""(278071.0, 324416.0]""",1
370761.0,"""(324416.0, 370761.0]""",0
417106.0,"""(370761.0, 417106.0]""",0


In [33]:
nodes_count.tail(10)#.head(10)

node,count
u32,u32
166419,97420
2650,100871
166129,107526
71520,117759
214198,129851
71546,136767
71511,146887
71514,185301
151453,307435


**Как видно из таблиц выше:**
- Всего существует 4 823 локаций, медианое кол-во объявлений в локациях - 217, максимальное кол-во объявлений в одной локации - 4.3 млн
- Общее кол-во категорий товара - 53, медианное кол-во объявлений в категориях - 167к, максимальное кол-во объявлений в одной категории - 3.69 млн
- Общее кол-во групп товаров - 408к, медианное кол-во объявлений в группе - 3, максимальное кол-во объявлений в одной группе - 463к

## Изучим и опишем данные из таблицы df_event

In [34]:
df_event.sort(by='event').describe()

statistic,event,is_contact
str,f64,f64
"""count""",19.0,19.0
"""null_count""",0.0,0.0
"""mean""",9.631579,0.684211
"""std""",6.048053,0.477567
"""min""",0.0,0.0
"""25%""",5.0,0.0
"""50%""",10.0,1.0
"""75%""",15.0,1.0
"""max""",19.0,1.0


In [35]:
df_event['is_contact'].value_counts()

is_contact,count
i64,u32
0,6
1,13


**df_event** - 20 событий:
- event - код события от 0 до 19,
- is_contact - контактное ли событие - 6 не контактных и 13 контактных.

# Подготовка моделей

In [36]:
EVAL_DAYS_TRESHOLD = 14

In [37]:
# Устанавливаем временной порог
treshhold = df_clickstream['event_date'].max() - timedelta(days=EVAL_DAYS_TRESHOLD)
treshhold

datetime.datetime(2025, 2, 9, 0, 0)

In [38]:
# Поведение пользователя до этого порога - трейн. после - валидационная
df_train = df_clickstream.filter(df_clickstream['event_date']<= treshhold)
df_eval = df_clickstream.filter(df_clickstream['event_date']> treshhold)[['cookie', 'node', 'event']]
df_train.shape, df_eval.shape

((45631770, 7), (23174382, 3))

Трейн - активность каждого пользователя ранее 2х недель назад. Эвал - активность каждого пользователя в последние 2 недели (взаимодействие с группами).

In [39]:
# Оставляем в df_eval данные по активности пользователей,
# Которые не имеют соответствий в df_train по указанным ключам
# Используя 'cookie' и 'node' (пользователя и группу товара) в качестве ключей
# anti - чтобы выбрать только те строки из df_eval, которых нет в df_train по cookie и node
df_eval = df_eval.join(df_train, on=['cookie', 'node'], how='anti')
df_eval

cookie,node,event
i64,u32,i64
1,196744,17
1,48631,17
1,267694,17
1,196909,17
1,402072,17
…,…,…
149993,195297,17
149993,195034,17
149995,115733,17


In [40]:
# Оставляем в df_eval только те строки,
# Где значение в столбце 'event' (событие клика) присутствует в уникальных событиях из df_event,
# Которые соответствуют условию is_contact == 1
df_eval = df_eval.filter(
    pl.col('event').is_in(
        df_event.filter(pl.col('is_contact')==1)['event'].unique()
    )
)

In [41]:
df_event.filter(pl.col('is_contact')==1)['event'].unique()

event
i64
0
1
2
4
5
…
13
14
15


In [42]:
df_eval

cookie,node,event
i64,u32,i64
86,986,10
122,107096,10
133,332870,15
182,220074,15
184,51162,10
…,…,…
149705,230737,10
149866,251805,15
149866,251805,10


In [43]:
# В df_eval оставляем только те строки,
# Где значение в столбце 'cookie' и 'node' присутствует в уникальных значениях из df_train
# То есть осталвяем только пользователей и их группы, которые есть и в трейне, и в тесте.
# То есть оставляем только записи о клиентах, если 14 или более дней назад такой интерес был 

df_eval = df_eval.filter(
    pl.col('cookie').is_in(df_train['cookie'].unique())  
    # в df_eval оставляем строки, где 'cookie' (пользователи) есть в уникальных значениях 'cookie' из df_train
).filter(
    pl.col('node').is_in(df_train['node'].unique())  
    # в df_eval оставляем строки, где 'node' (интерес) есть в уникальных значениях 'node' из df_train
)

In [44]:
# Оставляем уникальные комбинации пользователей и их групп
df_eval = df_eval.unique(['cookie', 'node'])

# Обучение моделей

## ALS (baseline)

In [45]:
def get_als_pred(users, nodes, user_to_pred):
    # Получаем уникальные идентификаторы пользователей и узлов (объявлений)
    user_ids = users.unique().to_list()
    item_ids = nodes.unique().to_list()
        
    # Создаем словари для сопоставления идентификаторов пользователей и узлов с их индексами
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
    index_to_item_id = {v: k for k, v in item_id_to_index.items()}
    
    # Заменяем идентификаторы пользователей и узлов на их индексы
    rows = users.replace_strict(user_id_to_index).to_list()
    cols = nodes.replace_strict(item_id_to_index).to_list()
    
    # Создаем значение для разреженной матрицы (в данном случае все значения равны 1)
    values = [1] * len(users)
    
    # Создаем разреженную матрицу, представляющую взаимодействия пользователей и узлов
    sparse_matrix = csr_matrix((values, (rows, cols)), shape=(len(user_ids), len(item_ids)))
    
    # Инициализируем модель ALS (Alternating Least Squares) с заданными параметрами
    model = implicit.als.AlternatingLeastSquares(iterations=10, factors=60)
    # Обучаем модель на разреженной матрице
    model.fit(sparse_matrix)
    
    # Получаем индексы пользователей, для которых нужно сделать рекомендации
    user4pred = np.array([user_id_to_index[i] for i in user_to_pred])
    
    # Генерируем рекомендации для указанных пользователей
    recommendations, scores = model.recommend(user4pred, sparse_matrix[user4pred], N=40, filter_already_liked_items=True)
    
    # Создаем DataFrame для хранения рекомендаций, идентификаторов узлов и оценок
    df_pred = pl.DataFrame(
        {
            'node': [
                [index_to_item_id[i] for i in i] for i in recommendations.tolist()  
                # Преобразуем индексы обратно в идентификаторы узлов
            ], 
            'cookie': list(user_to_pred),  # Сохраняем идентификаторы пользователей
            'scores': scores.tolist()  # Сохраняем оценки для каждой рекомендации
        }
    )
    
    # Разворачиваем DataFrame, чтобы каждая строка содержала одну рекомендацию
    df_pred = df_pred.explode(['node', 'scores'])
    
    return df_pred  # Возвращаем DataFrame с рекомендациями


In [46]:
from scipy.sparse import csr_matrix
import numpy as np
import implicit


users = df_train["cookie"]
nodes = df_train["node"]
eval_users = df_eval['cookie'].unique().to_list()

df_pred = get_als_pred(users, nodes, eval_users)
df_pred

C:\Users\kantoshina\anaconda3\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

node,cookie,scores
i64,i64,f64
115834,0,0.91071
116118,0,0.859938
116123,0,0.785304
214234,0,0.737184
214235,0,0.719916
…,…,…
152034,149998,0.377191
152043,149998,0.373568
152122,149998,0.373537


## popular (baseline)

In [47]:
def get_popular(df):
    # Группируем данные по 'node', считаем количество 'cookie' для каждого узла,
    # сортируем по количеству 'cookie' и берем 40 самых популярных узлов
    popular_node = df.group_by('node').agg(pl.col('cookie').count()).sort('cookie').tail(40)['node'].to_list()
    
    # Создаем DataFrame, где 'node' - это список популярных узлов, 
    # а 'cookie' - это список пользователей (eval_users)
    df_pred_pop = pl.DataFrame({'node': [popular_node for i in range(len(eval_users))], 'cookie': eval_users})
    
    # Разворачиваем столбец 'node', чтобы каждая строка соответствовала одному узлу
    df_pred_pop = df_pred_pop.explode('node')
    
    # Возвращаем полученный DataFrame
    return df_pred_pop

# Вызываем функцию get_popular с DataFrame df_train и сохраняем результат в train_pop
train_pop = get_popular(df_train)
train_pop

node,cookie
i64,i64
151614,0
24,0
159206,0
130596,0
71524,0
…,…
71511,149998
71546,149998
71514,149998


# Посчитаем метрики

In [48]:
def recall_at(df_true, df_pred, k=40):
    return  df_true[['node', 'cookie']].join(
        df_pred.group_by('cookie').head(k).with_columns(value=1)[['node', 'cookie', 'value']], 
        how='left',
        on = ['cookie', 'node']
    ).select(
        [pl.col('value').fill_null(0), 'cookie']
    ).group_by(
        'cookie'
    ).agg(
        [
            pl.col('value').sum()/pl.col(
                'value'
            ).count()
        ]
    )['value'].mean()

In [49]:
recall_at(df_eval, df_pred, k=40)

0.15068172242702435

In [50]:
recall_at(df_eval, train_pop, k=40)

0.058067308552970216

In [51]:
# Для проверки перед сабмитом На этапе eval в baseline ноутбучке можно использовать функцию

def check_recall_at(df_solution: pl.DataFrame, df_pred: pl.DataFrame, k=40):
    assert df_pred.group_by(['cookie']).agg(pl.col('node').count())['node'].max() <41 , 'send more then 40 nodes per cookie'
    assert 'node' in df_pred.columns, 'node columns does not exist'
    assert 'cookie' in df_pred.columns, 'cookie columns does not exist'
    assert df_pred.with_columns(v = 1).group_by(['cookie','node']).agg(pl.col('v').count())['v'].max() == 1 , 'more then 1 cookie-node pair'
    assert df_pred['cookie'].dtype == pl.Int64, 'cookie must be int64'
    assert df_pred['node'].dtype == pl.Int64, 'node must be int64'
    
    return  df_solution[['node', 'cookie']].join(
        df_pred.group_by('cookie').head(k).with_columns(value=1)[['node', 'cookie', 'value']], 
        how='left',
        on = ['cookie', 'node']
    ).select(
        [pl.col('value').fill_null(0), 'cookie']
    ).group_by(
        'cookie'
    ).agg(
        [
            pl.col('value').sum()/pl.col(
                'value'
            ).count()
        ]
    )['value'].mean()

def main(solution_path: str, prediction_path: str, stage: int):
    return check_recall_at(pl.read_csv(solution_path).filter(stage=stage), pl.read_csv(prediction_path))

# SUMBIT

In [52]:
users = df_clickstream["cookie"]
nodes = df_clickstream["node"]
test_users = df_test_users['cookie'].unique().to_list()

df_pred = get_als_pred(users, nodes, test_users )


  0%|          | 0/10 [00:00<?, ?it/s]

In [53]:
df_pred.write_csv('prediction.csv')

In [54]:
df_pred

node,cookie,scores
i64,i64,f64
243177,1,1.161905
115820,1,1.118542
214339,1,1.108312
152705,1,1.094247
243178,1,1.052569
…,…,…
116123,149999,0.500894
130636,149999,0.496801
120504,149999,0.495711
